# 다중 레이블 분류 (Multi-label Classification)

이 노트북은 하나의 샘플이 여러 개의 레이블(클래스)을 동시에 가질 수 있는 **다중 레이블 분류 (Multi-label Classification)** 문제를 다룹니다. 이는 전통적인 단일 레이블 분류(하나의 샘플이 하나의 클래스에만 속함)나 다중 클래스 분류(하나의 샘플이 여러 클래스 중 하나에 속함)와는 다릅니다.

## 다중 레이블 분류란?
다중 레이블 분류는 각 샘플이 0개, 1개 또는 그 이상의 레이블을 가질 수 있는 분류 문제입니다. 예를 들어:

- **영화 장르 예측**: 하나의 영화가 '코미디', '액션', '로맨스' 등 여러 장르에 동시에 속할 수 있습니다.
- **이미지 태깅**: 한 장의 사진에 '고양이', '나무', '하늘' 등 여러 객체가 동시에 존재할 수 있습니다.
- **문서 분류**: 하나의 문서가 '정치', '경제', '사회' 등 여러 주제를 동시에 다룰 수 있습니다.

## 학습 과정 개요:
1.  `scikit-learn`을 사용하여 다중 레이블 분류를 위한 가상 데이터셋 생성
2.  생성된 데이터를 훈련 세트와 테스트 세트로 분할
3.  `MultiOutputClassifier`와 `SVC`(서포트 벡터 머신)를 사용하여 모델 구성 및 훈련
4.  다중 레이블 분류에 적합한 평가 지표(Hamming Loss, Jaccard Score, F1-Score 등)를 사용하여 모델 성능 평가

### 1. 라이브러리 임포트

In [ ]:
import numpy as np  # 수치 계산을 위한 라이브러리
from sklearn.svm import SVC  # 서포트 벡터 분류기 (Support Vector Classifier)
from sklearn.multioutput import MultiOutputClassifier  # 다중 레이블 출력을 처리하는 래퍼(wrapper) 분류기
from sklearn.datasets import make_multilabel_classification  # 다중 레이블 분류용 가상 데이터셋 생성 함수
from sklearn.model_selection import train_test_split  # 훈련 및 테스트 데이터 분할 함수
from sklearn.metrics import hamming_loss, jaccard_score, f1_score  # 모델 성능 평가 지표

### 2. 다중 레이블 데이터 생성

`make_multilabel_classification` 함수는 다중 레이블 분류 문제에 적합한 가상의 데이터셋을 생성합니다. 이 함수는 특성(features) 행렬 `X`와 이진 레이블(binary labels) 행렬 `y`를 반환합니다.

- `n_samples`: 생성할 샘플(데이터 포인트)의 총 수입니다.
- `n_features`: 각 샘플이 가지는 특성(독립 변수)의 수입니다.
- `n_classes`: 전체 가능한 레이블(클래스)의 총 종류 수입니다. 예를 들어, 영화 장르 예측에서 '액션', '코미디', '드라마' 등 모든 가능한 장르의 수입니다.
- `n_labels`: 각 샘플이 평균적으로 가질 레이블의 수입니다. 이 값이 클수록 샘플당 더 많은 레이블이 할당됩니다.
- `random_state`: 재현 가능한 결과를 위해 난수 생성 시드를 고정합니다.

In [ ]:
# 100개의 샘플, 각 샘플은 20개의 특성을 가짐
# 전체 5가지 종류의 레이블이 존재하며, 각 샘플은 평균적으로 2개의 레이블을 가짐
X, y = make_multilabel_classification(n_samples=100, n_features=20, n_classes=5,
                                      n_labels=2, random_state=42)

print("특성(X) 데이터 형태:", X.shape) # (샘플 수, 특성 수)
print("레이블(y) 데이터 형태:", y.shape) # (샘플 수, 전체 레이블 종류 수)
print("\n첫 5개 샘플의 레이블 (이진 행렬):")
print(y[:5])

# y의 각 행은 하나의 샘플에 대한 레이블을 나타내는 이진 벡터입니다.
# 예를 들어, [1, 0, 1, 0, 0]은 첫 번째와 세 번째 레이블이 해당 샘플에 할당되었음을 의미합니다.

### 3. 훈련 및 테스트 세트 분리

모델의 일반화 성능을 평가하기 위해 전체 데이터셋을 훈련 세트(training set)와 테스트 세트(test set)로 분리하는 것은 머신러닝의 표준적인 절차입니다.

- `X_train`, `y_train`: 모델을 학습시키는 데 사용되는 데이터입니다.
- `X_test`, `y_test`: 학습된 모델의 성능을 평가하는 데 사용되는, 모델이 이전에 보지 못한 데이터입니다.
- `test_size`: 테스트 세트의 비율을 지정합니다 (예: 0.3은 전체 데이터의 30%를 테스트 세트로 사용).
- `random_state`: 데이터 분할 시 무작위성을 제어하여, 동일한 `random_state` 값을 사용하면 항상 동일한 분할 결과를 얻을 수 있습니다. 이는 실험의 재현성을 보장합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"훈련 데이터: {X_train.shape} (특성), {y_train.shape} (레이블)")
print(f"테스트 데이터: {X_test.shape} (특성), {y_test.shape} (레이블)")

### 4. 모델 구성 및 훈련

다중 레이블 분류를 위한 모델을 구성하고 훈련합니다. 여기서는 `MultiOutputClassifier`와 `SVC`를 조합하여 사용합니다.

## `MultiOutputClassifier` 상세 설명

`MultiOutputClassifier`는 다중 레이블 분류 문제를 해결하기 위한 전략 중 하나인 **이진 관련성 (Binary Relevance)** 접근 방식을 구현한 Scikit-learn의 래퍼(wrapper) 클래스입니다. 이 방식은 다중 레이블 문제를 여러 개의 독립적인 이진 분류 문제로 분해합니다.

**작동 방식:**
1.  **문제 분해**: `MultiOutputClassifier`는 다중 레이블 데이터셋의 각 레이블(클래스)에 대해 독립적인 단일 이진 분류기를 훈련시킵니다.
    - 예를 들어, 5개의 레이블이 있다면, 5개의 별도 이진 분류기(각 레이블의 존재 여부를 예측)를 훈련합니다.
2.  **독립적인 예측**: 각 이진 분류기는 해당 레이블의 존재 여부만을 예측합니다. 다른 레이블과의 상관관계는 고려하지 않습니다.
3.  **결과 통합**: 모든 이진 분류기의 예측 결과를 모아서 최종 다중 레이블 예측을 생성합니다.

**장점:**
- **단순성**: 구현이 간단하고 이해하기 쉽습니다.
- **유연성**: 어떤 단일 이진 분류기(예: 로지스틱 회귀, SVM, 결정 트리 등)든 `base_estimator`로 사용할 수 있습니다.
- **병렬 처리**: 각 레이블에 대한 분류기가 독립적이므로, `n_jobs` 파라미터를 사용하여 병렬로 훈련 및 예측을 수행할 수 있어 대규모 데이터셋에서 효율적입니다.

**단점:**
- **레이블 간 상관관계 무시**: 각 레이블을 독립적으로 처리하기 때문에, 레이블들 사이에 존재하는 잠재적인 상관관계(예: 영화가 '액션'이면서 '코미디'일 확률이 높다)를 모델이 학습하지 못합니다. 이는 성능 저하로 이어질 수 있습니다.
- **확장성 문제**: 레이블의 수가 매우 많아지면, 훈련해야 할 분류기의 수도 많아져 계산 비용이 증가할 수 있습니다.

## `SVC` (Support Vector Classifier) 상세 설명

`SVC`는 **서포트 벡터 머신 (Support Vector Machine, SVM)** 알고리즘을 기반으로 한 분류기입니다. SVM은 분류, 회귀, 이상치 탐지 등에 사용되는 강력한 지도 학습 모델입니다.

**핵심 아이디어:**
SVM의 목표는 서로 다른 클래스의 데이터 포인트들을 가장 잘 분리하는 **최적의 결정 경계(Optimal Hyperplane)**를 찾는 것입니다. 이 결정 경계는 각 클래스의 가장 가까운 데이터 포인트(서포트 벡터)로부터 가장 큰 마진(margin)을 가지도록 설정됩니다.

**주요 파라미터:**
- `kernel='linear'`: 선형 커널을 사용합니다. 이는 데이터가 선형적으로 분리 가능하다고 가정하고, 직선(2D) 또는 평면(3D 이상) 형태의 결정 경계를 찾습니다. 데이터가 복잡하거나 비선형적일 경우 'rbf'(방사형 기저 함수)와 같은 다른 커널을 사용할 수 있습니다.
- `probability=True`: `predict_proba` 메서드를 사용하여 클래스별 확률을 추정할 수 있도록 합니다. 이 과정은 교차 검증을 통해 수행되므로 훈련 시간이 길어질 수 있습니다. `predict` 메서드만 사용할 경우 `False`로 설정하여 성능을 향상시킬 수 있습니다.
- `random_state`: `probability=True`일 때 확률 추정 과정에 사용되는 난수 생성 시드를 고정합니다.

**`n_jobs=-1`**: `MultiOutputClassifier`에서 `n_jobs=-1`로 설정하면, 사용 가능한 모든 CPU 코어를 활용하여 각 레이블에 대한 분류기 훈련을 병렬로 수행합니다. 이는 훈련 시간을 단축시키는 데 매우 효과적입니다.

In [ ]:
# 각 레이블을 예측할 기본 모델로 SVC를 설정합니다.
# kernel='linear': 선형 결정 경계를 사용합니다.
# probability=True: 예측 확률을 계산할 수 있도록 설정합니다 (훈련 시간 증가 가능).
base_svm = SVC(kernel='linear', probability=True, random_state=42)

# MultiOutputClassifier로 SVC를 감싸 다중 레이블 문제에 적용합니다.
# n_jobs=-1은 가능한 모든 CPU 코어를 사용하여 각 레이블에 대한 분류기 훈련을 병렬로 진행하라는 의미입니다.
multi_label_svm = MultiOutputClassifier(base_svm, n_jobs=-1)

print("모델 훈련 시작...")
multi_label_svm.fit(X_train, y_train)
print("모델 훈련 완료.")

### 5. 예측

훈련된 `multi_label_svm` 모델을 사용하여 테스트 데이터 `X_test`에 대한 레이블을 예측합니다. `predict` 메서드는 각 샘플에 대해 예측된 레이블의 이진 행렬을 반환합니다.

In [ ]:
# 테스트 데이터에 대한 레이블 예측 수행
y_pred = multi_label_svm.predict(X_test)

print("실제 레이블 (첫 5개 샘플):")
print(y_test[:5])
print("\n예측된 레이블 (첫 5개 샘플):")
print(y_pred[:5])

### 6. 모델 평가

다중 레이블 분류는 단일 레이블 분류와는 다른, 특화된 평가 지표를 사용합니다. 이는 하나의 샘플이 여러 개의 정답 레이블을 가질 수 있기 때문입니다.

#### 6.1. 햄밍 손실 (Hamming Loss)

**개념**: 햄밍 손실은 전체 레이블 중에서 잘못 예측된 레이블의 비율을 나타냅니다. 즉, 실제 레이블과 예측 레이블이 다른 위치의 개수를 전체 레이블 개수로 나눈 값입니다.

**계산**: 각 샘플에 대해 실제 레이블과 예측 레이블이 일치하지 않는 레이블의 수를 세고, 이를 전체 샘플의 전체 레이블 수로 나눈 값입니다.

**해석**: 햄밍 손실은 **0에 가까울수록** 모델 성능이 좋다는 것을 의미합니다. 0이면 모든 레이블을 완벽하게 예측했다는 뜻입니다.

**예시**: 
샘플 1의 실제 레이블: `[1, 0, 1]`
샘플 1의 예측 레이블: `[1, 1, 1]`

이 경우, 두 번째 레이블(인덱스 1)에서 불일치(`0` vs `1`)가 발생했습니다. 전체 3개 레이블 중 1개가 잘못 예측되었으므로, 이 샘플의 햄밍 손실은 1/3 입니다. 모든 샘플에 대해 이 값을 계산하여 평균을 냅니다.

In [ ]:
h_loss = hamming_loss(y_test, y_pred)
print(f"Hamming Loss: {h_loss:.4f}")

#### 6.2. 자카드 점수 (Jaccard Score / Jaccard Similarity Coefficient)

**개념**: 자카드 점수(또는 자카드 유사도)는 두 집합 간의 유사도를 측정하는 지표입니다. 다중 레이블 분류에서는 실제 레이블 집합과 예측 레이블 집합 간의 유사도를 나타냅니다. 이는 **교집합의 크기를 합집합의 크기로 나눈 값**입니다.

**계산**: 
$$ J(A, B) = \frac{|A \cap B|}{|A \cup B|} $$
여기서 $A$는 실제 레이블 집합, $B$는 예측 레이블 집합입니다.

**해석**: 자카드 점수는 **1에 가까울수록** 모델 성능이 좋다는 것을 의미합니다. 1이면 실제 레이블과 예측 레이블이 완벽하게 일치한다는 뜻입니다.

- `average='samples'`: 각 샘플에 대해 자카드 점수를 개별적으로 계산한 후, 이 점수들의 평균을 반환합니다. 이는 샘플별 예측의 정확도를 평가하는 데 유용합니다. 이 경우, 한 샘플에 대한 모든 레이블을 **완벽하게** 예측한 경우에만 높은 점수를 받기 때문에 'Exact Match Ratio'와 유사하게 엄격한 지표로 사용될 수 있습니다.

In [ ]:
# average='samples'는 각 샘플에 대해 실제 레이블과 예측 레이블 간의 Jaccard 유사도를 계산한 후 평균을 냅니다.
jaccard_similarity = jaccard_score(y_test, y_pred, average='samples')
print(f"Jaccard Score (samples average): {jaccard_similarity:.4f}")

#### 6.3. F1-점수 (F1-Score)

**개념**: F1-점수는 정밀도(Precision)와 재현율(Recall)의 조화 평균(harmonic mean)입니다. 이는 특히 클래스 불균형이 있는 경우에 유용한 지표입니다. F1-점수는 정밀도와 재현율 모두가 높을 때 높은 값을 가집니다.

**정밀도 (Precision)**: 모델이 '예측한 양성' 중에서 실제로 '양성'인 비율입니다. (오탐(False Positive)을 줄이는 데 중요)
$$ Precision = \frac{TP}{TP + FP} $$

**재현율 (Recall)**: 실제 '양성' 중에서 모델이 '양성'으로 올바르게 예측한 비율입니다. (미탐(False Negative)을 줄이는 데 중요)
$$ Recall = \frac{TP}{TP + FN} $$

**F1-점수**: 
$$ F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall} $$

**해석**: F1-점수는 **1에 가까울수록** 모델 성능이 좋다는 것을 의미합니다.

**다중 레이블에서의 F1-점수 평균화 전략:**
다중 레이블 분류에서는 여러 레이블에 대한 F1-점수를 하나의 값으로 통합하기 위해 다양한 평균화(averaging) 전략을 사용합니다.

- `average='micro'`: 
    - **작동 방식**: 모든 레이블에 대한 True Positives (TP), False Positives (FP), False Negatives (FN)를 각각 합산한 후, 이 합산된 값들을 사용하여 전체 정밀도와 재현율을 계산하고 최종 F1-점수를 구합니다.
    - **특징**: 레이블 간의 불균형이 심할 때 유용합니다. 다수 클래스의 성능에 더 큰 영향을 받습니다. 전체 시스템의 성능을 평가하는 데 적합합니다.
    - **예시**: 전체 예측 중 올바르게 예측된 레이블의 비율을 중요하게 볼 때 사용합니다.

- `average='macro'`: 
    - **작동 방식**: 각 개별 레이블에 대해 F1-점수를 각각 계산한 후, 이 계산된 F1-점수들의 산술 평균을 냅니다.
    - **특징**: 모든 레이블을 동등하게 중요하다고 간주합니다. 레이블 간의 불균형이 있을 경우, 소수 클래스의 성능이 전체 점수에 큰 영향을 미칠 수 있습니다. 각 레이블의 성능이 고르게 중요할 때 사용합니다.
    - **예시**: 모든 영화 장르(액션, 코미디, 드라마 등)에 대해 모델이 고르게 잘 예측하는지 확인하고 싶을 때 사용합니다.

- `average='weighted'` (참고): 
    - **작동 방식**: 각 개별 레이블의 F1-점수를 계산한 후, 각 레이블의 실제 샘플 수(support)에 비례하여 가중 평균을 냅니다.
    - **특징**: 레이블 불균형을 고려하면서도, 더 많은 샘플을 가진 레이블의 성능에 더 큰 가중치를 줍니다.

- `average='samples'` (참고): 
    - **작동 방식**: 각 샘플에 대해 F1-점수를 계산한 후, 이 점수들의 평균을 냅니다. 이는 `jaccard_score`의 `average='samples'`와 유사하게 샘플별 정확도를 평가합니다.
    - **특징**: 각 샘플의 예측이 얼마나 정확한지에 초점을 맞춥니다.

In [ ]:
# micro: 전체 TP, FP, FN을 합산하여 계산 (레이블 불균형에 덜 민감하며, 전체적인 예측 정확도에 가깝습니다)
f1_micro = f1_score(y_test, y_pred, average='micro')
print(f"Micro F1-score: {f1_micro:.4f}")

# macro: 각 레이블에 대한 F1-score를 계산한 후 평균 (모든 레이블을 동등하게 중요하게 간주하며, 소수 레이블의 성능도 반영합니다)
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-score: {f1_macro:.4f}")

# weighted: 각 레이블의 실제 샘플 수(support)에 따라 가중치를 부여하여 평균 (레이블 불균형을 고려하면서도 다수 레이블에 더 가중치를 줍니다)
f1_weighted = f1_score(y_test, y_pred, average='weighted')
print(f"Weighted F1-score: {f1_weighted:.4f}")

# samples: 각 샘플에 대한 F1-score를 계산한 후 평균 (샘플별 예측의 정확도에 초점을 맞춥니다)
f1_samples = f1_score(y_test, y_pred, average='samples')
print(f"Samples F1-score: {f1_samples:.4f}")